In [ ]:
from pyspark import SparkContext
import pickle
import math
import random

# Initialize SparkContext
try:
    sc
except NameError:
    sc = SparkContext("local[*]", "ManualDecisionTree")

# Set high parallelism for large dataset
sc._conf.set("spark.default.parallelism", "256")

# Load and parse the training data
lines = sc.textFile("/kaggle/input/testmet/train.csv", minPartitions=256)
header = lines.first()
data = lines.filter(lambda line: line != header)

# Preprocessing parameters
LONGITUDE_MIN, LONGITUDE_MAX = -74.3, -73.7
LATITUDE_MIN, LATITUDE_MAX = 40.5, 41.0
PASSENGER_MIN, PASSENGER_MAX = 1, 9
TRIP_DURATION_MIN, TRIP_DURATION_MAX = 60, 36000  # 1 minute to 10 hours

# Parse and preprocess training data 
def parse_train_line(line):
    cols = line.split(",")
    try:
        vendor_id = float(cols[1])
        passenger_count = float(cols[4])
        pickup_longitude = float(cols[5])
        pickup_latitude = float(cols[6])
        dropoff_longitude = float(cols[7])
        dropoff_latitude = float(cols[8])
        trip_duration = float(cols[10])

        # Filter outliers
        if not (LONGITUDE_MIN <= pickup_longitude <= LONGITUDE_MAX and
                LONGITUDE_MIN <= dropoff_longitude <= LONGITUDE_MAX and
                LATITUDE_MIN <= pickup_latitude <= LATITUDE_MAX and
                LATITUDE_MIN <= dropoff_latitude <= LATITUDE_MAX):
            return None
        if not (PASSENGER_MIN <= passenger_count <= PASSENGER_MAX):
            return None
        if not (TRIP_DURATION_MIN <= trip_duration <= TRIP_DURATION_MAX):
            return None
        if vendor_id not in (1, 2):
            return None

        features = [vendor_id, passenger_count, pickup_longitude, pickup_latitude, dropoff_longitude, dropoff_latitude]
        return (features, trip_duration)
    except:
        return None

parsed_data = data.map(parse_train_line).filter(lambda x: x is not None).persist()

# Compute feature min/max once
num_features = 6
feature_stats = []
for i in range(num_features):
    feature_rdd = parsed_data.map(lambda x: x[0][i])
    min_val, max_val = feature_rdd.min(), feature_rdd.max()
    feature_stats.append((min_val, max_val))

# Split into training and validation sets
train_rdd, val_rdd = parsed_data.randomSplit([0.8, 0.2], seed=42)
train_rdd.persist()
val_rdd.persist()

# Parse test data
test_lines = sc.textFile("/kaggle/input/testmet/test.csv", minPartitions=256)
test_header = test_lines.first().strip()
test_data = test_lines.filter(lambda line: line.strip() != test_header)

def parse_test_line(line):
    cols = line.split(",")
    try:
        id_val = cols[0]
        vendor_id = float(cols[1])
        passenger_count = float(cols[3])
        pickup_longitude = float(cols[4])
        pickup_latitude = float(cols[5])
        dropoff_longitude = float(cols[6])
        dropoff_latitude = float(cols[7])

        # Filter outliers
        if not (LONGITUDE_MIN <= pickup_longitude <= LONGITUDE_MAX and
                LONGITUDE_MIN <= dropoff_longitude <= LONGITUDE_MAX and
                LATITUDE_MIN <= pickup_latitude <= LATITUDE_MAX and
                LATITUDE_MIN <= dropoff_latitude <= LATITUDE_MAX):
            return None
        if not (PASSENGER_MIN <= passenger_count <= PASSENGER_MAX):
            return None
        if vendor_id not in (1, 2):
            return None

        features = [vendor_id, passenger_count, pickup_longitude, pickup_latitude, dropoff_longitude, dropoff_latitude]
        return (id_val, features)
    except:
        return None

test_data_parsed = test_data.map(parse_test_line).filter(lambda x: x is not None).persist()

# Decision tree functions
def calculate_mse_and_count(data_rdd):
    zero_value = (0.0, 0.0, 0)
    seq_op = lambda acc, x: (acc[0] + x[1], acc[1] + x[1]**2, acc[2] + 1)
    comb_op = lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1], acc1[2] + acc2[2])
    stats = data_rdd.aggregate(zero_value, seq_op, comb_op)
    sum_y, sum_y2, count = stats
    if count == 0:
        return 0.0, 0
    mean = sum_y / count
    variance = (sum_y2 / count) - mean**2 if count > 0 else 0.0
    return variance, count

def find_best_split(data_rdd, feature_idx, min_val, max_val, num_thresholds=10):
    if min_val >= max_val:
        return None, float("inf")

    # Sample thresholds randomly for speed
    def get_thresholds():
        if max_val == min_val:
            return [min_val]
        return [min_val + random.random() * (max_val - min_val) for _ in range(num_thresholds)]

    thresholds = get_thresholds()
    total_count = data_rdd.count()
    total_stats = data_rdd.aggregate(
        (0.0, 0.0, 0),
        lambda acc, x: (acc[0] + x[1], acc[1] + x[1]**2, acc[2] + 1),
        lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1], acc1[2] + acc2[2])
    )
    total_sum_y, total_sum_y2, total_count_agg = total_stats
    total_variance = (total_sum_y2 / total_count_agg - (total_sum_y / total_count_agg)**2) if total_count_agg > 0 else 0.0

    best_threshold = None
    best_mse = float("inf")

    for threshold in thresholds:
        left_stats = data_rdd.filter(lambda x: x[0][feature_idx] <= threshold).aggregate(
            (0.0, 0.0, 0),
            lambda acc, x: (acc[0] + x[1], acc[1] + x[1]**2, acc[2] + 1),
            lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1], acc1[2] + acc2[2])
        )
        left_sum_y, left_sum_y2, left_count = left_stats
        right_sum_y = total_sum_y - left_sum_y
        right_sum_y2 = total_sum_y2 - left_sum_y2
        right_count = total_count_agg - left_count

        if left_count < 1000 or right_count < 1000:  # Increased min node size
            continue

        left_mse = (left_sum_y2 / left_count - (left_sum_y / left_count)**2) if left_count > 0 else 0
        right_mse = (right_sum_y2 / right_count - (right_sum_y / right_count)**2) if right_count > 0 else 0
        total_mse = (left_mse * left_count + right_mse * right_count) / total_count_agg

        if total_mse < best_mse:
            best_mse = total_mse
            best_threshold = threshold

    # Variance reduction check
    if best_mse >= total_variance * 0.95:  # Looser threshold for speed
        return None, float("inf")

    return best_threshold, best_mse

def build_decision_tree(data_rdd, depth=0, max_depth=20):
    count = data_rdd.count()
    print(f"Depth {depth}, count {count}")

    if count < 2000 or depth >= max_depth:  # Increased min total size
        stats = data_rdd.aggregate(
            (0.0, 0),
            lambda acc, x: (acc[0] + x[1], acc[1] + 1),
            lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1])
        )
        sum_y, count = stats
        return {"leaf": True, "prediction": sum_y / count if count > 0 else 0.0}

    # Evaluate features sequentially to avoid RDD serialization
    best_feature = None
    best_threshold = None
    best_mse = float("inf")

    for idx in range(num_features):
        threshold, mse = find_best_split(data_rdd, idx, feature_stats[idx][0], feature_stats[idx][1])
        if threshold is not None and mse < best_mse:
            best_mse = mse
            best_feature = idx
            best_threshold = threshold

    if best_feature is None:
        stats = data_rdd.aggregate(
            (0.0, 0),
            lambda acc, x: (acc[0] + x[1], acc[1] + 1),
            lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1])
        )
        sum_y, count = stats
        return {"leaf": True, "prediction": sum_y / count if count > 0 else 0.0}

    left_rdd = data_rdd.filter(lambda x: x[0][best_feature] <= best_threshold).persist()
    right_rdd = data_rdd.filter(lambda x: x[0][best_feature] > best_threshold).persist()
    left_tree = build_decision_tree(left_rdd, depth + 1, max_depth)
    right_tree = build_decision_tree(right_rdd, depth + 1, max_depth)
    left_rdd.unpersist()
    right_rdd.unpersist()

    return {
        "leaf": False,
        "feature": best_feature,
        "threshold": best_threshold,
        "left": left_tree,
        "right": right_tree
    }

# Build the decision tree
tree = build_decision_tree(train_rdd, max_depth=4)

# Save the tree
with open("decision_tree_model.pkl", "wb") as f:
    pickle.dump(tree, f)
print("Decision tree model saved to decision_tree_model.pkl")

def predict(tree, features):
    if tree["leaf"]:
        return tree["prediction"]
    if features[tree["feature"]] <= tree["threshold"]:
        return predict(tree["left"], features)
    return predict(tree["right"], features)

# Evaluate on validation set
val_predictions = val_rdd.map(lambda x: (x[1], predict(tree, x[0])))
val_mse = val_predictions.map(lambda lp: (lp[0] - lp[1])**2).mean()
val_rmse = val_mse ** 0.5
print(f"Validation Mean Squared Error: {val_mse}")
print(f"Validation Root Mean Squared Error: {val_rmse}")

# Sample predictions on test data
sample_test = test_data_parsed.take(5)
print("Sample Predictions from test.csv:")
for id_val, features in sample_test:
    pred = predict(tree, features)
    print(f"ID: {id_val}, Features: {features}, Predicted trip_duration: {pred:.2f}")

# Save all test predictions
test_predictions = test_data_parsed.map(lambda x: (x[0], predict(tree, x[1])))
test_predictions.saveAsTextFile("test_predictions.txt")

# Clean up
train_rdd.unpersist()
val_rdd.unpersist()
test_data_parsed.unpersist()
parsed_data.unpersist()

# Stop SparkContext
sc.stop()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/12 09:08:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Depth 0, count 79374


Depth 1, count 75697


Depth 2, count 73735


Depth 2, count 1962


Depth 1, count 3677


Depth 2, count 2298


Depth 2, count 1379


Decision tree model saved to decision_tree_model.pkl


Validation Mean Squared Error: 295578.5206743891
Validation Root Mean Squared Error: 543.6713351597535
Sample Predictions from test.csv:
ID: id3004672, Features: [1.0, 1.0, -73.98812866210938, 40.73202896118164, -73.99017333984375, 40.7566795349121], Predicted trip_duration: 743.79
ID: id3505355, Features: [1.0, 1.0, -73.96420288085938, 40.67999267578125, -73.95980834960938, 40.65540313720703], Predicted trip_duration: 743.79
ID: id1217141, Features: [1.0, 1.0, -73.9974365234375, 40.73758316040039, -73.9861602783203, 40.729522705078125], Predicted trip_duration: 743.79
ID: id2150126, Features: [2.0, 1.0, -73.95606994628906, 40.77190017700195, -73.98642730712889, 40.73046875], Predicted trip_duration: 743.79
ID: id1598245, Features: [1.0, 1.0, -73.97021484375, 40.761474609375, -73.96150970458984, 40.755889892578125], Predicted trip_duration: 743.79
